In [ ]:
import os
import sys
import django
import pandas as pd
import plotly.express as px

In [ ]:
# Adicione o caminho até a pasta onde está o manage.py
sys.path.append(r'C:\Users\pedro\Documents\Python Scripts\Projeto_TCC\prototipo_tcc\src\mysite')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mysite.settings")
django.setup()

In [ ]:
from gest_financeira.models import Registros
from django.contrib.auth import get_user_model
from asgiref.sync import sync_to_async

User = get_user_model()

async def get_usuario_and_dados(user_id):
	usuario = await sync_to_async(User.objects.get)(id=user_id)
	dados = await sync_to_async(Registros.objects.filter)(id_user=usuario)
	return usuario, dados

usuario, dados = await get_usuario_and_dados(18)


In [ ]:
import pandas as pd

# 'dados' é um queryset, converta para lista de dicionários
dados_list = await sync_to_async(list)(dados.values())

# Crie o DataFrame
df_dados = pd.DataFrame(dados_list)

# Exiba o DataFrame (no notebook, só coloque df na última linha)
display(df_dados)


In [ ]:
resume_pie = df_dados[["category","amount","type_register"]]
resume_pie = resume_pie[resume_pie['type_register'] != 'entrada']
resume_pie.groupby("category", as_index=False)['amount'].sum()
display(resume_pie)
fig = px.pie(resume_pie,names='category',values='amount',title="Gasto por Categoria")
fig.update_layout(
    paper_bgcolor='#005F6B',  # Cor do fundo total (papel)
    plot_bgcolor='#005F6B',        # Cor do fundo dentro do gráfico
    font_color='white'
)
fig.show()


In [ ]:
fig = px.pie(
    resume_pie,
    names='category',
    values='amount',
    title='Gasto por Categoria',
    color_discrete_sequence=px.colors.sequential.RdBu  # Escolhe uma paleta de cores diferente
)

fig.update_traces(
    textposition='inside',  # texto dentro da fatia
    textinfo='percent+label',  # mostra % e nome
)

fig.update_layout(
    title_font_size=20,
    legend_title_text='Categorias',
    legend=dict(
        orientation='v',
        yanchor='middle',
        y=0.5,
        xanchor='right',
        x=1.2
    )
)

fig.show()